In [21]:
!pip install spacy
!pip install huggingface
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bionlp13cg_md-0.5.1.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz
!pip install https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl

# Requirements

In [1]:
import pandas as pd
import spacy

drug = spacy.load("en_core_med7_lg")
gene = spacy.load("en_ner_bionlp13cg_md")
dis_chem = spacy.load('en_ner_bc5cdr_md')

# Dataset Preprocessing

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Gene entities/500_test_documents.csv', sep='|', header=None).rename(columns={0:'Doc', 1:'Type', 2:'Content'})

In [7]:
data.head(5)

,Doc,Type,Content
0,Doc0,t,Association between promoter -1607 polymorphis...
1,Doc0,a,BACKGROUND: Matrix metalloproteinases (MMPs) a...
2,Doc1,t,Ginsenoside Rg1 restores the impairment of lea...
3,Doc1,a,"Rg1, as a ginsenoside extracted from Panax gin..."
4,Doc2,t,Lack of association between ADRA2B-4825 gene i...


In [11]:
data.pivot(columns='Type', values='Content')

Type,a,t
0,NaN,Association between promoter -1607 polymorphis...
1,BACKGROUND: Matrix metalloproteinases (MMPs) a...,NaN
2,NaN,Ginsenoside Rg1 restores the impairment of lea...
3,"Rg1, as a ginsenoside extracted from Panax gin...",NaN
4,NaN,Lack of association between ADRA2B-4825 gene i...
...,...,...
995,OBJECTIVE: This study determined the antihyper...,NaN
996,NaN,Attenuation of methamphetamine-induced nigrost...
997,Immunological activation has been proposed to ...,NaN
998,NaN,Mechanisms of hypertension induced by nitric o...


# Pipeline

In [7]:
data['Gene'] = data.Content.apply(lambda x: [ent.text for ent in gene(x).ents
                                              if (label:=ent.label_) == 'GENE_OR_GENE_PRODUCT'])

data['Disease'] = data.Content.apply(lambda x: [ent.text for ent in dis_chem(x).ents
                                              if (label:=ent.label_) == 'DISEASE'])

data['Drug'] = data.Content.apply(lambda x: [ent.text for ent in drug(x).ents
                                              if (label:=ent.label_) == 'DRUG'])

In [8]:
data.to_csv('/content/drive/MyDrive/Gene entities/Data with genes.csv', index=False)